---
# 12. Shortest Path
|Problem|Dfficulty|Link|
|--------|--------|-----------|
|787. Cheapest Flights Within K Stops | <span style="color:yellow">Medium</span> | https://leetcode.com/problems/cheapest-flights-within-k-stops/description |
|1334. Find the City With the Smallest Number of Neighbors at a Threshold Distance | <span style="color:yellow">Medium</span> | https://leetcode.com/problems/find-the-city-with-the-smallest-number-of-neighbors-at-a-threshold-distance/description | 
|1368. Minimum Cost to Make at Least One Valid Path in a Grid | <span style="color:red">Hard</span>  | https://leetcode.com/problems/minimum-cost-to-make-at-least-one-valid-path-in-a-grid/description | 
|2203. Minimum Weighted Subgraph With the Required Paths | <span style="color:red">Hard</span> | https://leetcode.com/problems/minimum-weighted-subgraph-with-the-required-paths/description |
|2642. Design Graph With Shortest Path Calculator | <span style="color:red">Hard</span>  | https://leetcode.com/problems/design-graph-with-shortest-path-calculator/description |

---
## 787. Cheapest Flights Within K Stops

# Intuition
This can be tackled using dynamic programming (DP) with modified Bellman-Ford algorithm in order to keep track of the minimum cost at each step up to `k` stops.

# Approach
1. **Dynamic Programming Initialization**:
   - Use a 2D DP array `dp` where `dp[i][j]` represents the minimum cost to reach node `i` with at most `j` stops.
   - Initialize the `dp` array to infinity (`INF`) to represent unreachable states. The size of the array is determined by the number of nodes `n` and the maximum stops `k`.

2. **Set Up Base Case**:
   - The cost to reach the source node with 0 stops is 0 (`dp[src][0] = 0`).

3. **Filling the DP Table**:
   - Iterate over the number of stops from 0 to `k`.
   - For each stop, update the DP table based on the flight information. This involves relaxing the edges similar to the Bellman-Ford algorithm:
     - For each flight from `u` to `v` with cost `w`, if the current cost to reach `u` with `s` stops is less than infinity, update the cost to reach `v` with `s+1` stops.

4. **Find the Minimum Cost**:
   - The result is the minimum cost to reach the destination node `dst` with up to `k` stops. Iterate through the last row of the DP table for the destination node.

# Complexity

## Time complexity: `O(n * k * |E|)`, where `n` is the number of nodes, `k` is the maximum number of stops, and `|E|` is the number of edges (flights). 

## Space complexity: `O(n * k)`, for storing the DP table.

# Code

```cpp 
#include <bits/stdc++.h>

using namespace std;

class Solution {
private:
    const int MAX_N = 1e2 + 2;
    const int MAX_K = 1e2 + 2;
    const int INF = 1e9;
public:
    int findCheapestPrice(int n, vector<vector<int>>& flights, int src, int dst, int k) {
        vector<vector<int>> dp(k + 2, vector<int>(n, INF));
        dp[0][src] = 0;

        for (int i = 1; i <= k + 1; ++i) {
            dp[i][src] = 0;
            for (const auto& flight : flights) {
                int from = flight[0], to = flight[1], price = flight[2];
                if (dp[i - 1][from] != INF) {
                    dp[i][to] = min(dp[i][to], dp[i - 1][from] + price);
                }
            }
        }

        int answer = INF;
        for (int i = 1; i <= k + 1; ++i) {
            answer = min(answer, dp[i][dst]);
        }
        return answer == INF ? -1 : answer;
    }
};
```

---
## 1334. Find the City With the Smallest Number of Neighbors at a Threshold Distance

# Intuition
Use the Floyd-Warshall algorithm to compute the shortest paths between all pairs of cities and then determine the city that meets the criteria.

# Approach
1. **Initialize Distance Matrix**:
   - Create a 2D distance matrix `dist` of size `n x n` initialized to infinity (`INT_MAX`), except for the diagonal which is initialized to 0 (distance from a city to itself).

2. **Fill Distance Matrix with Given Edges**:
   - For each edge, update the distance matrix with the given weights for the directly connected cities.

3. **Floyd-Warshall Algorithm**:
   - Apply the Floyd-Warshall algorithm to compute the shortest paths between all pairs of cities:
     - The outermost loop iterates through each intermediate city `k`.
     - Iterate through each pair of cities `i` and `j`, updating the shortest distance between them considering the intermediate city `k`.

4. **Determine the Result City**:
   - For each city, count the number of other cities reachable within the given distance threshold.
   - Track the city with the smallest number of reachable cities. 
        - If there is a tie, choose the city with the greatest index.

# Complexity

## Time complexity: `O(n^3)`
## Space complexity: `O(n^2)`

# Code

```cpp
class Solution {
public:
    int findTheCity(int n, vector<vector<int>>& edges, int distanceThreshold) {
        // Initialize the distance matrix with infinity
        vector<vector<int>> dist(n, vector<int>(n, INT_MAX));
        
        // Distance from a city to itself is 0
        for (int i = 0; i < n; ++i) {
            dist[i][i] = 0;
        }
        
        // Fill the distance matrix with given edges
        for (auto& edge : edges) {
            int u = edge[0], v = edge[1], weight = edge[2];
            dist[u][v] = weight;
            dist[v][u] = weight;
        }
        
        for (int k = 0; k < n; ++k) {
            for (int i = 0; i < n; ++i) {
                for (int j = 0; j < n; ++j) {
                    if (dist[i][k] != INT_MAX && dist[k][j] != INT_MAX) {
                        dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j]);
                    }
                }
            }
        }
        
        // Find the city with the smallest number of reachable cities within the distance threshold
        int minReachable = n;
        int resultCity = -1;
        
        for (int i = 0; i < n; ++i) {
            int reachableCities = 0;
            for (int j = 0; j < n; ++j) {
                if (i != j && dist[i][j] <= distanceThreshold) {
                    reachableCities++;
                }
            }
            
            if (reachableCities < minReachable || (reachableCities == minReachable && i > resultCity)) {
                minReachable = reachableCities;
                resultCity = i;
            }
        }
        
        return resultCity;
    }
};

```

---
## 1368. Minimum Cost to Make at Least One Valid Path in a Grid

# Intuition
A priority queue (min-heap) can be used to always expand the least costly path first, similar to Dijkstra's algorithm.

# Approach
1. **Initialize Structures**:
   - Use a 2D vector `dist` to store the minimum cost to reach each cell, initialized to `INT_MAX`.
   - Use a priority queue `pq` to store the cells to be processed, with their current path costs.
   - Start from the top-left cell with an initial cost of 0.

2. **Processing the Queue**:
   - While the queue is not empty, extract the cell with the minimum cost.
   - For each of the four possible directions (right, left, down, up), calculate the cost to move to the adjacent cell.
   - If moving in the intended direction (as indicated by the cell's value) results in no additional cost, otherwise, the cost is incremented by 1.
   - If the new calculated cost to reach an adjacent cell is less than the current recorded cost, update the cost and push the adjacent cell to the queue.

3. **Termination**:
   - The process continues until the bottom-right cell is reached.
   - If the queue is exhausted without reaching the bottom-right cell, return -1 (though for a valid input, it should always be reachable).

# Complexity

## Time complexity
- `O(m * n * log(m * n))`, where `m` is the number of rows and `n` is the number of columns in the grid.

## Space complexity
- `O(m * n)`

# Code

```cpp  
class Solution {
public:
    int minCost(vector<vector<int>>& grid) {
        int m = grid.size();
        int n = grid[0].size();

        vector<vector<int>> dist(m, vector<int>(n, INT_MAX));
        priority_queue<pair<int, pair<int, int>>, vector<pair<int, pair<int, int>>>, greater<pair<int, pair<int, int>>>> pq;

        pq.push({0, {0, 0}});
        dist[0][0] = 0;

        while (!pq.empty()) {
            auto [cost, node] = pq.top();
            pq.pop();

            int row = node.first;
            int col = node.second;

            if (row == m - 1 && col == n - 1) {
                return cost;
            }

            for (int i = 0; i < 4; i++) {
                int x = row + dx[i];
                int y = col + dy[i];

                if (x >= 0 && x < m && y >= 0 && y < n) {
                    int newCost = cost + checkCost(dx[i], dy[i], grid[row][col]);

                    if (newCost < dist[x][y]) {
                        dist[x][y] = newCost;
                        pq.push({newCost, {x, y}});
                    }
                }
            }
        }

        return -1;
    }

private:
    const int dx[4] = {1, -1, 0, 0};
    const int dy[4] = {0, 0, 1, -1};
    const int valMap[4] = {3, 4, 1, 2};

    int checkCost(int dx, int dy, int val) {
        for (int i = 0; i < 4; ++i) {
            if (this->dx[i] == dx && this->dy[i] == dy && this->valMap[i] == val) return 0;
        }
        return 1;
    }
};

```

---
## 2203. Minimum Weighted Subgraph With the Required Paths

# Intuition
Use Dijkstra's algorithm to compute the shortest paths from each source and to the destination.

# Approach
1. **Graph Representation**:
   - Represent the graph using adjacency lists. Also, construct a reverse graph to efficiently compute paths to the destination.

2. **Dijkstra's Algorithm**:
   - Implement Dijkstra's algorithm to compute the shortest paths from a given source to all other nodes. Use a priority queue to ensure that the shortest unprocessed path is always expanded first.

3. **Compute Shortest Paths**:
   - Use Dijkstra's algorithm to compute the shortest paths:
     - From `src1` to all nodes.
     - From `src2` to all nodes.
     - From `dest` to all nodes in the reverse graph.

4. **Combine Paths**:
   - For each node, calculate the combined path weight as the sum of the shortest paths from `src1` and `src2` to the node, and from the node to `dest`.
   - Track the minimum combined weight across all nodes.

5. **Return the Result**:
   - If a valid path is found, return the minimum combined weight. Otherwise, return `-1` indicating no valid path exists.

# Complexity

## Time complexity
- `O((V + E) log V)`, where `V` is the number of vertices and `E` is the number of edges.

## Space complexity
- `O(V + E)`, for storing the adjacency lists and the distances.

```cpp  
class Solution {
public:
    vector<long long> dijkstra(int n, vector<vector<pair<int, int>>>& adj, int start) {
        vector<long long> dist(n, LLONG_MAX);
        dist[start] = 0;
        priority_queue<pair<long long, int>, vector<pair<long long, int>>, greater<pair<long long, int>>> pq;
        pq.push({0, start});
        
        while (!pq.empty()) {
            auto [d, u] = pq.top();
            pq.pop();
            if (d > dist[u]) continue;
            
            for (auto& [v, w] : adj[u]) {
                if (dist[u] + w < dist[v]) {
                    dist[v] = dist[u] + w;
                    pq.push({dist[v], v});
                }
            }
        }
        
        return dist;
    }

    long long minimumWeight(int n, vector<vector<int>>& edges, int src1, int src2, int dest) {
        vector<vector<pair<int, int>>> adj(n), rev_adj(n);
        
        for (auto& edge : edges) {
            int u = edge[0], v = edge[1], w = edge[2];
            adj[u].emplace_back(v, w);
            rev_adj[v].emplace_back(u, w);
        }
        
        vector<long long> distFromSrc1 = dijkstra(n, adj, src1);
        vector<long long> distFromSrc2 = dijkstra(n, adj, src2);
        vector<long long> distToDest = dijkstra(n, rev_adj, dest);
        
        long long minWeight = LLONG_MAX;
        for (int i = 0; i < n; ++i) {
            if (distFromSrc1[i] != LLONG_MAX && distFromSrc2[i] != LLONG_MAX && distToDest[i] != LLONG_MAX) {
                minWeight = min(minWeight, distFromSrc1[i] + distFromSrc2[i] + distToDest[i]);
            }
        }
        
        return minWeight == LLONG_MAX ? -1 : minWeight;
    }
};
```



---
## 2642. Design Graph With Shortest Path Calculator

# Intuition
Implement a graph class that can dynamically add edges and compute the shortest path between two nodes using Dijkstra's algorithm.

# Approach
1. **Graph Initialization**:
   - Store the number of nodes `n` and an adjacency list `adj` to represent the graph. Initialize the adjacency list with the given edges.

2. **Add Edge**:
   - Implement a method `addEdge` to add a new edge to the graph. This method updates the adjacency list with the new edge.

3. **Shortest Path Calculation**:
   - Use a priority queue to always expand the least costly path first.
   - Maintain a distance array `dist` where `dist[i]` represents the shortest distance from `node1` to node `i`.
   - Initialize all distances to infinity (`INT_MAX`) except for the starting node which is set to 0.
   - Extracting the next node from the priority queue, and for each of its neighbors, 
        - if a shorter path is found, update their distances 
   - If the distance to `node2` remains infinity, return -1 indicating no path exists; otherwise, return the shortest distance.

# Complexity

## Time complexity: `O((V + E) log V)`, where `V` is the number of vertices and `E` is the number of edges. 

## Space complexity: `O(V + E)`

```cpp
typedef pair<int, int> pii;

class Graph {
private:
    int n;
    vector<vector<pii> adj;

public:
    Graph(int n, vector<vector<int>>& edges) : n(n), adj(n) {
        for (auto& edge : edges) {
            adj[edge[0]].emplace_back(edge[1], edge[2]);
        }
    }
    
    void addEdge(vector<int> edge) {
        adj[edge[0]].emplace_back(edge[1], edge[2]);
    }
    
    int shortestPath(int node1, int node2) {
        vector<int> dist(n, INT_MAX);
        dist[node1] = 0;
        
        priority_queue<pii, vector<pii>, greater<pii>> pq;
        pq.emplace(0, node1);
        
        while (!pq.empty()) {
            auto [currentDist, u] = pq.top();
            pq.pop();
            
            if (currentDist > dist[u]) continue;
            
            for (auto& [v, weight] : adj[u]) {
                if (dist[u] + weight < dist[v]) {
                    dist[v] = dist[u] + weight;
                    pq.emplace(dist[v], v);
                }
            }
        }
        
        return dist[node2] == INT_MAX ? -1 : dist[node2];
    }
};
```